In [11]:
!pip install -q langchain langchain-community langchain-openai langchain-text-splitters \
               sentence-transformers faiss-cpu pypdf tiktoken
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from langchain_community.vectorstores import FAISS
from langchain.embeddings.base import Embeddings

from sentence_transformers import SentenceTransformer
from pypdf import PdfReader


In [12]:
# 📌 Step 1: Upload your PDF
from google.colab import files
uploaded = files.upload()

# 📌 Step 2: Get the filename automatically
pdf_path = list(uploaded.keys())[0]
print("Using PDF file:", pdf_path)

# 📌 Step 3: Load and split the PDF
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
raw_docs = loader.load()

print("Loaded pages:", len(raw_docs))

# Splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

docs = splitter.split_documents(raw_docs)

print("Number of chunks:", len(docs))


Saving _Final__Build__Train__and_Imagine_with_LLMs.pdf to _Final__Build__Train__and_Imagine_with_LLMs (1).pdf
Using PDF file: _Final__Build__Train__and_Imagine_with_LLMs (1).pdf
Loaded pages: 3
Number of chunks: 10


In [13]:
class SBertEmbeddings(Embeddings):
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, show_progress_bar=True).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

embedding_function = SBertEmbeddings()


In [14]:
db = FAISS.from_documents(docs, embedding_function)
print("FAISS index built!")

db.save_local("index_store")
print("Index saved to index_store/")

db = FAISS.load_local("index_store", embedding_function, allow_dangerous_deserialization=True)
print("Index loaded!")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index built!
Index saved to index_store/
Index loaded!
